https://www.kaggle.com/c/competitive-data-science-predict-future-sales/


In [66]:
import pandas as pd
import numpy as np

In [67]:
data = pd.read_csv("./data/sales_train.csv")

In [68]:
data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [69]:
data = data.groupby(["date_block_num", "shop_id", "item_id"])["item_cnt_day"].sum().reset_index(name="item_cnt_month")

In [70]:
unique_shop_id = data["shop_id"].unique()
unique_item_id = data["item_id"].unique()
unique_date_block_num = data["date_block_num"].unique()

In [71]:
from itertools import product

In [72]:
date_block_shop_item_grid = pd.DataFrame([(shop_id, item_id, date_block) for shop_id, item_id, date_block in product(unique_shop_id, unique_item_id, unique_date_block_num)], columns=["shop_id", "item_id", "date_block_num"])

In [73]:
date_block_shop_item_grid = date_block_shop_item_grid.merge(data, on=["shop_id", "item_id", "date_block_num"], how="left")

In [74]:
date_block_shop_item_grid.fillna({"item_cnt_month": 0}, inplace=True)

In [75]:
date_block_shop_item_grid.head()

,shop_id,item_id,date_block_num,item_cnt_month
0,0,32,0,6.0
1,0,32,1,10.0
2,0,32,2,0.0
3,0,32,3,0.0
4,0,32,4,0.0


In [76]:
folds = [(list(range(25,31)),[31]), (list(range(26,32)),[32]), (list(range(27,33)),[33])]

In [77]:
from sklearn.metrics import mean_squared_error

## previuos period

In [114]:
metrics = []

In [115]:
for train_range, validation_range in  folds:
    train_data = date_block_shop_item_grid.loc[date_block_shop_item_grid['date_block_num'].isin(train_range)]
    valid_data =  date_block_shop_item_grid.loc[date_block_shop_item_grid['date_block_num'].isin(validation_range)]
    train_data['prev_value'] = train_data.groupby(['shop_id','item_id'])['item_cnt_month'].shift()
    train_data['prev_value'] = train_data['prev_value'].fillna(value=0)
    valid_data['prev_value'] = valid_data.groupby(['shop_id','item_id'])['item_cnt_month'].shift()
    valid_data['prev_value'] = valid_data['prev_value'].fillna(value=0)
    clipped_pred =  np.clip(valid_data['prev_value'], 0, 20)
    clipped_valid = np.clip(valid_data['item_cnt_month'], 0, 20)
    error = mean_squared_error(clipped_valid, clipped_pred)
    metrics.append(error)

/tmp/ipykernel_427048/2660704190.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['prev_value'] = train_data.groupby(['shop_id','item_id'])['item_cnt_month'].shift()
/tmp/ipykernel_427048/2660704190.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['prev_value'] = train_data['prev_value'].fillna(value=0)
/tmp/ipykernel_427048/2660704190.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [116]:
metrics

[0.19874657984439248, 0.2342535271548891, 0.24714311918191406]

In [117]:
train_data.columns

Index(['shop_id', 'item_id', 'date_block_num', 'item_cnt_month', 'prev_value'], dtype='object')

In [118]:
train_data.groupby(['shop_id', 'item_id', 'date_block_num']).head(10)

,shop_id,item_id,date_block_num,item_cnt_month,prev_value
27,0,32,27,0.0,0.0
28,0,32,28,0.0,0.0
29,0,32,29,0.0,0.0
30,0,32,30,0.0,0.0
31,0,32,31,0.0,0.0
...,...,...,...,...,...
44486274,36,15925,28,0.0,0.0
44486275,36,15925,29,0.0,0.0
44486276,36,15925,30,0.0,0.0
44486277,36,15925,31,0.0,0.0


In [119]:
groups = train_data.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_month'].mean()

In [78]:
train_data['prev_value'] = train_data.groupby(['shop_id','item_id'])['item_cnt_month'].shift()

In [79]:
train_data[(train_data.shop_id == 2) & (train_data.item_id == 32)].sort_values(by=['shop_id', 'item_id', 'date_block_num']).head(300)

,shop_id,item_id,date_block_num,item_cnt_month,prev_value
1482903,2,32,27,0.0,NaN
1482904,2,32,28,0.0,0.0
1482905,2,32,29,0.0,0.0
1482906,2,32,30,0.0,0.0
1482907,2,32,31,1.0,0.0
1482908,2,32,32,0.0,1.0
1482909,2,32,33,0.0,0.0


In [80]:
train_data[train_data.prev_value > 1]

,shop_id,item_id,date_block_num,item_cnt_month,prev_value
1486478,2,1523,32,0.0,2.0
1487630,2,1905,28,1.0,2.0
1488448,2,2252,30,1.0,2.0
1488450,2,2252,32,1.0,5.0
1488719,2,2308,29,3.0,3.0
...,...,...,...,...,...
41494449,34,4839,33,0.0,3.0
41494483,34,4840,33,0.0,8.0
41494551,34,5041,33,0.0,2.0
41495095,34,3558,33,1.0,3.0


In [81]:
test = pd.read_csv("./data/sales_test.csv")

In [82]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [83]:
test['date_block_num'] = 34

In [84]:
test.head()

,ID,shop_id,item_id,date_block_num
0,0,5,5037,34
1,1,5,5320,34
2,2,5,5233,34
3,3,5,5232,34
4,4,5,5268,34


In [85]:
prev_train = date_block_shop_item_grid[date_block_shop_item_grid.date_block_num == 33]
prev_train

,shop_id,item_id,date_block_num,item_cnt_month
33,0,32,33,0.0
67,0,33,33,0.0
101,0,35,33,0.0
135,0,43,33,0.0
169,0,51,33,0.0
...,...,...,...,...
44486143,36,12733,33,0.0
44486177,36,13092,33,0.0
44486211,36,16797,33,0.0
44486245,36,18060,33,0.0


In [86]:
concated = pd.concat([prev_train, test])
concated

,shop_id,item_id,date_block_num,item_cnt_month,ID
33,0,32,33,0.0,NaN
67,0,33,33,0.0,NaN
101,0,35,33,0.0,NaN
135,0,43,33,0.0,NaN
169,0,51,33,0.0,NaN
...,...,...,...,...,...
214195,45,18454,34,NaN,214195.0
214196,45,16188,34,NaN,214196.0
214197,45,15757,34,NaN,214197.0
214198,45,19648,34,NaN,214198.0


In [87]:
concated['prediction'] = concated.groupby(['shop_id','item_id'])['item_cnt_month'].shift()
concated[concated.prediction > 1]

,shop_id,item_id,date_block_num,item_cnt_month,ID,prediction
6,5,5041,34,NaN,6.0,2.0
10,5,4806,34,NaN,10.0,3.0
14,5,4870,34,NaN,14.0,2.0
15,5,4872,34,NaN,15.0,6.0
18,5,4892,34,NaN,18.0,2.0
...,...,...,...,...,...,...
211419,45,9401,34,NaN,211419.0,2.0
211513,45,11161,34,NaN,211513.0,2.0
211605,45,3617,34,NaN,211605.0,2.0
211985,45,7877,34,NaN,211985.0,2.0


In [88]:
concated[(concated.shop_id == 2) & (concated.item_id == 32)].sort_values(by=['shop_id', 'item_id', 'date_block_num']).head(300)

,shop_id,item_id,date_block_num,item_cnt_month,ID,prediction
1482909,2,32,33,0.0,NaN,NaN
20995,2,32,34,NaN,20995.0,0.0


In [89]:
submission = concated[concated.date_block_num == 34][['ID', 'prediction']]

In [90]:
submission.rename(columns={"prediction": "item_cnt_month"}, inplace=True)
submission.reset_index(drop=True, inplace=True)

In [91]:
submission['item_cnt_month'] = submission['item_cnt_month'].fillna(value=0)
submission['item_cnt_month'] = np.clip(submission['item_cnt_month'], 0, 20)
submission

,ID,item_cnt_month
0,0.0,0.0
1,1.0,0.0
2,2.0,1.0
3,3.0,0.0
4,4.0,0.0
...,...,...
214195,214195.0,1.0
214196,214196.0,0.0
214197,214197.0,0.0
214198,214198.0,0.0


In [92]:
submission.astype({'ID': 'int32'}).to_csv('submission_sales.csv', index=False)

## mean n periods

In [93]:
period_to_predict = max(date_block_shop_item_grid.date_block_num) +1
period_to_predict

34

In [94]:
date_block_shop_item_grid.date_block_num.nunique()

34

In [95]:
n = 6
max_periods_num = date_block_shop_item_grid.date_block_num.nunique() - 1
if n > max_periods_num:
    n = max_periods_num

In [96]:
start_period = max_periods_num - n
start_period

27

In [97]:
train_data = date_block_shop_item_grid[date_block_shop_item_grid.date_block_num >= start_period]

In [98]:
train_data

,shop_id,item_id,date_block_num,item_cnt_month
27,0,32,27,0.0
28,0,32,28,0.0
29,0,32,29,0.0
30,0,32,30,0.0
31,0,32,31,0.0
...,...,...,...,...
44486275,36,15925,29,0.0
44486276,36,15925,30,0.0
44486277,36,15925,31,0.0
44486278,36,15925,32,0.0


In [99]:
concated = pd.concat([train_data, test])
concated

,shop_id,item_id,date_block_num,item_cnt_month,ID
27,0,32,27,0.0,NaN
28,0,32,28,0.0,NaN
29,0,32,29,0.0,NaN
30,0,32,30,0.0,NaN
31,0,32,31,0.0,NaN
...,...,...,...,...,...
214195,45,18454,34,NaN,214195.0
214196,45,16188,34,NaN,214196.0
214197,45,15757,34,NaN,214197.0
214198,45,19648,34,NaN,214198.0


In [100]:
#concated['prediction'] = concated.groupby(['shop_id','item_id'])['item_cnt_month'].transform('mean')
concated['prediction'] = concated.groupby(['shop_id','item_id'], sort=False)['item_cnt_month'].transform(lambda x: x.expanding().mean())
concated

,shop_id,item_id,date_block_num,item_cnt_month,ID,prediction
27,0,32,27,0.0,NaN,0.000000
28,0,32,28,0.0,NaN,0.000000
29,0,32,29,0.0,NaN,0.000000
30,0,32,30,0.0,NaN,0.000000
31,0,32,31,0.0,NaN,0.000000
...,...,...,...,...,...,...
214195,45,18454,34,NaN,214195.0,0.142857
214196,45,16188,34,NaN,214196.0,0.000000
214197,45,15757,34,NaN,214197.0,0.000000
214198,45,19648,34,NaN,214198.0,0.000000


In [101]:
concated[(concated.shop_id == 2) & (concated.item_id == 32)].sort_values(by=['shop_id', 'item_id', 'date_block_num']).head(300)

,shop_id,item_id,date_block_num,item_cnt_month,ID,prediction
1482903,2,32,27,0.0,NaN,0.000000
1482904,2,32,28,0.0,NaN,0.000000
1482905,2,32,29,0.0,NaN,0.000000
1482906,2,32,30,0.0,NaN,0.000000
1482907,2,32,31,1.0,NaN,0.200000
1482908,2,32,32,0.0,NaN,0.166667
1482909,2,32,33,0.0,NaN,0.142857
20995,2,32,34,NaN,20995.0,0.142857


In [102]:
concated[concated.prediction > 20]

,shop_id,item_id,date_block_num,item_cnt_month,ID,prediction
1829839,2,20949,27,66.0,NaN,66.000000
1829840,2,20949,28,44.0,NaN,55.000000
1829841,2,20949,29,45.0,NaN,51.666667
1829842,2,20949,30,42.0,NaN,49.250000
1829843,2,20949,31,8.0,NaN,41.000000
...,...,...,...,...,...,...
193854,41,3731,34,NaN,193854.0,24.857143
204054,39,3731,34,NaN,204054.0,23.857143
204460,39,20949,34,NaN,204460.0,84.714286
209154,45,3731,34,NaN,209154.0,22.428571


In [103]:
submission = concated[concated.date_block_num == 34][['ID', 'prediction']]
submission.rename(columns={"prediction": "item_cnt_month"}, inplace=True)
submission.reset_index(drop=True, inplace=True)
submission['item_cnt_month'] = submission['item_cnt_month'].fillna(value=0)
submission['item_cnt_month'] = np.clip(submission['item_cnt_month'], 0, 20)
submission

,ID,item_cnt_month
0,0.0,1.000000
1,1.0,0.000000
2,2.0,1.428571
3,3.0,0.142857
4,4.0,0.000000
...,...,...
214195,214195.0,0.142857
214196,214196.0,0.000000
214197,214197.0,0.000000
214198,214198.0,0.000000


In [104]:
submission.astype({'ID': 'int32'}).to_csv('submission_sales_mean.csv', index=False)

## medain n periods

In [105]:
period_to_predict = max(date_block_shop_item_grid.date_block_num) +1
n = 7
max_periods_num = date_block_shop_item_grid.date_block_num.nunique() - 1
if n > max_periods_num:
    n = max_periods_num
start_period = max_periods_num - n
train_data = date_block_shop_item_grid[date_block_shop_item_grid.date_block_num >= start_period]
concated = pd.concat([train_data, test])
#concated['prediction'] = concated.groupby(['shop_id','item_id'])['item_cnt_month'].transform('median')
concated['prediction'] = concated.groupby(['shop_id','item_id'], sort=False)['item_cnt_month'].transform(lambda x: x.expanding().median())

In [106]:
concated[(concated.shop_id == 2) & (concated.item_id == 32)].sort_values(by=['shop_id', 'item_id', 'date_block_num']).head(300)

,shop_id,item_id,date_block_num,item_cnt_month,ID,prediction
1482902,2,32,26,1.0,NaN,1.0
1482903,2,32,27,0.0,NaN,0.5
1482904,2,32,28,0.0,NaN,0.0
1482905,2,32,29,0.0,NaN,0.0
1482906,2,32,30,0.0,NaN,0.0
1482907,2,32,31,1.0,NaN,0.0
1482908,2,32,32,0.0,NaN,0.0
1482909,2,32,33,0.0,NaN,0.0
20995,2,32,34,NaN,20995.0,0.0


In [107]:
concated[concated.prediction > 1]

,shop_id,item_id,date_block_num,item_cnt_month,ID,prediction
1487628,2,1905,26,2.0,NaN,2.0
1487629,2,1905,27,2.0,NaN,2.0
1487630,2,1905,28,1.0,NaN,2.0
1487631,2,1905,29,1.0,NaN,1.5
1488716,2,2308,26,2.0,NaN,2.0
...,...,...,...,...,...,...
209985,45,16227,34,NaN,209985.0,1.5
210001,45,1384,34,NaN,210001.0,1.5
210079,45,4894,34,NaN,210079.0,2.0
211167,45,1556,34,NaN,211167.0,1.5


In [108]:
submission = concated[concated.date_block_num == 34][['ID', 'prediction']]
submission.rename(columns={"prediction": "item_cnt_month"}, inplace=True)
submission.reset_index(drop=True, inplace=True)
submission['item_cnt_month'] = submission['item_cnt_month'].fillna(value=0)
submission['item_cnt_month'] = np.clip(submission['item_cnt_month'], 0, 20)
submission

,ID,item_cnt_month
0,0.0,1.0
1,1.0,0.0
2,2.0,1.0
3,3.0,0.0
4,4.0,0.0
...,...,...
214195,214195.0,0.0
214196,214196.0,0.0
214197,214197.0,0.0
214198,214198.0,0.0


In [109]:
submission.astype({'ID': 'int32'}).to_csv('submission_sales_median.csv', index=False)